In [1]:
#import all the libraries
from io import StringIO
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import datetime
from pandas import read_csv
from pandas import DataFrame
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from matplotlib import pyplot

<ipython-input-1-d6ec1ead46c0>:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


If you want to use any of the datasets in the alteryx bucket, please use the following code:

In [ ]:
#This gets the alteryx output dataset:
url = 'https://capstoneprojectycbs299.s3.ca-central-1.amazonaws.com/masterdata.csv'

response = requests.get(url)

data = StringIO(response.text)
fire_dataset = pd.read_csv(data)

In [ ]:
#Create date column
grouped_dataset = fire_dataset
grouped_dataset['date'] = pd.to_datetime(dict(year=fire_dataset.INCIDENT_YEAR, month=fire_dataset.INCIDENT_MONTH, day=1))
#remove last 1 months of 2023
# 2023-09-01
grouped_dataset = grouped_dataset[grouped_dataset['date'] < '2023-09-01']

In [ ]:
#group dataset by date and caserne
column = {
    'NUMBER_OF_INCIDENTS':'sum',
    'ARROND_UNITS':'median',
    'ARROND_DETERIOATION_LOGEMENTS':'median',
    'ARROND_SALUBRITES':'median',
    'ARROND_PERMITS':'median',
    'ARROND_VACANCIES':'median',
    'ARROND_TREES':'median',
    'ARROND_RUELLES_VERTES':'median',
    'ARROND_AVG_CONSTRUCTION_YEAR':'median',
    'ARROND_EXTERMINATIONS':'median',
    'AVERAGE_TEMP':'median',
    'ARROND_AIDE_ALIMENTAIRE':'median',
    'ARROND_POPULATION':'median',
    'ARROND_INFRACTIONS_ENTRAINANT_MORT': 'median',
    'ARROND_INTRODUCTION': 'median',
    'AVERAGE_DISTANCE': 'median'
}
grouped_dataset = grouped_dataset.groupby(['CASERNE', 'date'], group_keys=False, as_index=False).agg(column)

In [ ]:
#sort by date
grouped_dataset = grouped_dataset.sort_values(by='date', ascending= True)

grouped_dataset.head()

,CASERNE,date,NUMBER_OF_INCIDENTS,ARROND_UNITS,ARROND_DETERIOATION_LOGEMENTS,ARROND_SALUBRITES,ARROND_PERMITS,ARROND_VACANCIES,ARROND_TREES,ARROND_RUELLES_VERTES,ARROND_AVG_CONSTRUCTION_YEAR,ARROND_EXTERMINATIONS,AVERAGE_TEMP,ARROND_AIDE_ALIMENTAIRE,ARROND_POPULATION,ARROND_INFRACTIONS_ENTRAINANT_MORT,ARROND_INTRODUCTION,AVERAGE_DISTANCE
3933,26,2005-01-01,10,66218.0,0.0,0.0,29.0,110.0,17806.0,269.0,1936.0,0.0,-10.516129,28.0,100390.0,0.0,0.0,8732.214935
9473,54,2005-01-01,1,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,-10.516129,0.0,21253.0,0.0,0.0,8751.908607
9295,53,2005-01-01,4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,-10.516129,1.0,19505.0,0.0,0.0,8752.401693
9276,52,2005-01-01,2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,-10.516129,1.0,19505.0,0.0,0.0,8755.531696
1716,15,2005-01-01,6,49880.0,0.0,0.0,20.0,50.0,21003.0,104.0,1973.0,0.0,-10.516129,16.5,71546.0,0.0,0.0,8730.194454


In [ ]:
#test
test_data = grouped_dataset[(grouped_dataset.CASERNE == 34)]
test_data.head()

,CASERNE,date,NUMBER_OF_INCIDENTS,ARROND_UNITS,ARROND_DETERIOATION_LOGEMENTS,ARROND_SALUBRITES,ARROND_PERMITS,ARROND_VACANCIES,ARROND_TREES,ARROND_RUELLES_VERTES,ARROND_AVG_CONSTRUCTION_YEAR,ARROND_EXTERMINATIONS,AVERAGE_TEMP,ARROND_AIDE_ALIMENTAIRE,ARROND_POPULATION,ARROND_INFRACTIONS_ENTRAINANT_MORT,ARROND_INTRODUCTION,AVERAGE_DISTANCE
5571,34,2005-01-01,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10.516129,0.0,165031.0,0.0,0.0,8734.057159
5572,34,2005-02-01,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.214286,0.0,165031.0,0.0,0.0,8734.057159
5573,34,2005-03-01,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.032258,0.0,165031.0,0.0,0.0,8734.057159
5574,34,2005-04-01,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.233333,0.0,165031.0,0.0,0.0,8734.057159
5575,34,2005-05-01,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.419355,0.0,165031.0,0.0,0.0,8734.057159


In [ ]:
#get casernes in a list to iterate through
caserne_list = fire_dataset.CASERNE.unique()

[43 49 42 35 28 78 34  4 46 27 61 54 56 64 65 30 16 26 33 15  3 23 48 45
 39 40 74  8 18 17 75 59 57 55 38 14 44 32 29 47 31 50 73 72 71 21 22 66
 67 19  5 10 25 20  9 41 37 76 53 63 13 77 51 58 62 52]


In [ ]:
#build date labels:
forecast_date_labels = ['2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
                        '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
                         '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
                         '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01',]

In [ ]:
from IPython.display import clear_output
forecast_Results = pd.DataFrame({"date":[],
                    "caserne":[],
                      "incidents": []});
steps_forecast = len(forecast_date_labels)
#iterate through the caserne:
for caserne in caserne_list:
  #filter the dataset for only this caserne:
  model_dataset = grouped_dataset[(grouped_dataset.CASERNE == caserne)]
  #define model:
  model = ARIMA(model_dataset['NUMBER_OF_INCIDENTS'], order=(10,1,0))
  model_fit = model.fit()
  forecast_output = model_fit.forecast(steps = steps_forecast)
  df2 = pd.DataFrame({"date":forecast_date_labels,
                      "caserne":[caserne] * steps_forecast,
                        "incidents": forecast_output})
  #build result array:
  forecast_Results = forecast_Results.append(df2, ignore_index = True)
  clear_output()

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this

In [ ]:
forecast_Results.head(30)

,date,caserne,incidents
0,2023-09-01,43.0,4.694473
1,2023-10-01,43.0,5.630797
2,2023-11-01,43.0,5.200977
3,2023-12-01,43.0,4.614909
4,2024-01-01,43.0,4.314467
5,2024-02-01,43.0,5.431944
6,2024-03-01,43.0,5.421682
7,2024-04-01,43.0,5.827577
8,2024-05-01,43.0,5.223694
9,2024-06-01,43.0,4.602057


In [ ]:
forecast_Results.to_csv('forecast_Results.csv', sep=',', index=False, encoding='utf-8')